In [59]:
#!pip install scenedetect
#!pip install opencv-python

In [60]:
from scenedetect import open_video, SceneManager
from scenedetect.detectors import ContentDetector

path = "input.mp4"

video = open_video(path)
manager = SceneManager()
manager.add_detector(ContentDetector(threshold=30.0))
manager.detect_scenes(video)

scenes = manager.get_scene_list()

### Export sous-titre des scenes

In [61]:
out = []

for i, (start, end) in enumerate(scenes, 1):
    out.append(str(i))
    out.append(f"{start.get_timecode()} --> {end.get_timecode()}")
    out.append(f"Scene {i}")
    out.append("")  # ligne vide entre blocs

scene_count = "\n".join(out)

In [62]:
srt_path = path + "_desciption.srt"

# Écriture du fichier SRT
with open(srt_path, "w", encoding="utf-8") as f:
    f.write(scene_count)

### Export 1 image / scene

In [63]:
import cv2

def save_frame_cv(input_path, frame_number, out_path):
    cap = cv2.VideoCapture(input_path)

    # aller directement à la frame demandée
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

    ok, frame = cap.read()
    if not ok:
        raise RuntimeError(f"Impossible de lire la frame {frame_number}.")
    
    cv2.imwrite(out_path, frame)
    cap.release()

In [64]:
from pathlib import Path

# enlever l'extension
folder = Path(path).with_suffix("")

# créer le dossier s'il n'existe pas
folder.mkdir(parents=True, exist_ok=True)

In [68]:
for i, (start, end) in enumerate(scenes, 1):
    middle = int((end.frame_num + start.frame_num)/2)
    
    image_path = folder / f"Scene {i}.jpg"

    save_frame_cv(path, middle, image_path)